# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('C:/Users/Константин/Python/main_task.xls')

In [ ]:
display(df)

In [ ]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

In [3]:
df.rename(columns={'Restaurant_id': 'restaurant_id',
                   'City': 'city', 'Cuisine Style': 'cuisine_style',
                   'Ranking': 'ranking', 'Rating': 'rating', 
                   'Price Range': 'price_range', 
                   'Number of Reviews': 'number_of_reviews', 
                   'Reviews': 'reviews', 'URL_TA': 'url_ta', 
                   'ID_TA': 'id_ta'
                  }, inplace=True)

In [ ]:
df.info()

### 1. City

Количество городов в представленном датасете - 31. Пропусков нет. Максимальное число ресторанов в одном городе - 5757 (Лондон), минимальное - 183 (Любляна). Представлены столичные города, а также второй-третий по величине город всех европейских регионов (запад, восток, север, юг). Причем все города находятся в составе экономически развитых государств с большим количеством туристов. 

Для целей будущего анализа создан словарь, сопоставляющий город с национальной кухней.

При подготовке модели данный столбец использовать не будем.

In [ ]:
len(df.city.value_counts())

In [ ]:
df.city.value_counts()

In [4]:
cities = df.city.unique()

In [5]:
city_dict = {'London': 'British', 'Paris': 'French', 
             'Madrid': 'Spanish', 'Barcelona': 'Spanish',
             'Berlin': 'German', 'Milan': 'Italy', 
             'Rome': 'Italy', 'Prague': 'Czech', 
             'Lisbon': 'Portuguese', 'Vienna': 'Austrian', 
             'Amsterdam': 'Dutch', 'Brussels': 'Belgian', 
             'Hamburg': 'German', 'Munich': 'German', 
             'Lyon': 'French', 'Stockholm': 'Swedish', 
             'Budapest': 'Hungarian', 'Warsaw': 'Polish', 
             'Dublin': 'Irish', 'Copenhagen': 'Danish', 
             'Athens': 'Greek', 'Edinburgh': 'Scottish', 
             'Zurich': 'Swiss', 'Oporto': 'Portuguese', 
             'Geneva': 'Swiss', 'Krakow': 'Polish', 
             'Oslo': 'Norwegian', 'Helsinki': 'Finland', 
             'Bratislava': 'Slovak', 'Luxembourg': 'Belgian', 
             'Ljubljana': 'Slovenian'
            }

### 2. Cuisine Style

Наименования представленных в ресторане стилей кухонь (125 наименований) могут отражать принадлежность к определенному региону или государству, а также виды еды.
Пропущенных значений - 9283 (23%). Наиболее распространенное число стилей кухонь - 1 (6698 ресторанов), 2 (5474), 3 (3969), 4 (3321), 5 (2672) и далее по убыванию. Максимальное число кухонь на один ресторан - 27, наблюдается в одном ресторане. Среднее число видов кухонь - 2.6.

Замена пропусков. Частично (40%) пустые значения заполнил в результате поиска ключевых слов в отзывах. Остальные пропуски заполнил исходя из национальной кухни в соответствии с городом расположения ресторана.

Перевод данного признака в цифровой вид осуществил через создание dummy-переменных. Для этого выделил 20 самых распространенных стилей. Также некоторые стили объединил в один: 'Vegetarian Friendly' и 'Vegan Options', 'Japanese' и 'Sushi', ('Bar', 'Pub', 'Wine Bar', 'Brew Pub', 'Gastropub'), ('Fast Food', 'Street Food, 'Delicatessen').

In [6]:
df.cuisine_style = df.cuisine_style\
    .map(lambda x: str(x)[1:][:-1])
df.cuisine_style = df.cuisine_style\
    .map(lambda x: re.sub(r"[']", "", x))

In [7]:
cuisines = set(df.cuisine_style.str.split(', ').sum())
cuisines_dict = dict(zip(list(cuisines),\
                [i[1:6].lower() for i in cuisines]))
# создано множество значений стилей кухонь и словарь: 
# ключи из множества стилей,значения - первые пять символов

In [8]:
cuisines.remove('a')

In [ ]:
a = pd.Series({x:df[df.cuisine_style.str.contains(x)]\
               .restaurant_id.count() for x in cuisines})\
               .sort_values(ascending = False)

In [ ]:
len(a)

In [ ]:
display(a.head(25))

In [9]:
df['cuisine_count'] = df.cuisine_style.str.split()\
                      .str.len()

In [10]:
df.cuisine_count.mean()

3.374075

In [11]:
def find_cuisine(x):
    list = []
    for k, v in cuisines_dict.items():
        if x.find(v) >= 0:
            list.append(k)
    return list
#ф-я поиска нахождения значений словаря в строке отзывов

In [12]:
df['reviews_cuisine'] = df.reviews.apply(find_cuisine)
#создал отдельный столбец с найдеными стилями

In [13]:
df.cuisine_style = np.select([df.cuisine_style == 'a'],\
            [df.reviews_cuisine], df.cuisine_style)
#в моем случае значения NaN отображаются как 'a', заменил

In [14]:
mask = df.cuisine_style.astype('str')\
                          .isin(['NaN', 'nan', '', '[]'])
df.loc[mask, 'cuisine_style'] = df.loc[mask, 'city']\
                                   .map(city_dict)
#замена пустых значений на национальные кухни по словарю

In [15]:
df.cuisine_style.value_counts()

[a]                                                                                                                    7135
Italian                                                                                                                1032
French                                                                                                                  805
[Bar, a]                                                                                                                731
Spanish                                                                                                                 695
                                                                                                                       ... 
Asian, Malaysian                                                                                                          1
Indian, Asian, Pakistani, Middle Eastern, Halal, Vegetarian Friendly                                                      1
Seafood,

Далее создаю dummy-признаки. Не получилось воспользоваться методом get_dummies, т.к. может быть несколько значений.

In [16]:
df['veget_friendly'] = df.cuisine_style.str.count('Vegetarian Friendly')
df['vegan_opt'] = df.cuisine_style.str.count('Vegan Options')
df['vegan'] = df['veget_friendly'] + df['vegan_opt']
df = df.drop(['veget_friendly', 'vegan_opt'], axis=1)

In [17]:
df['european'] = df.cuisine_style.str.count('European')

In [18]:
df['glutenfree'] = df.cuisine_style.str.count('Gluten Free Options')

In [19]:
df['mediterranean'] = df.cuisine_style.str.count('Mediterranean')

In [20]:
df['bar_s'] = df.cuisine_style.str.count('Bar')
df['pub'] = df.cuisine_style.str.count('Pub')
df['winebar'] = df.cuisine_style.str.count('Wine Bar')
df['brew_pub'] = df.cuisine_style.str.count('Brew Pub')
df['gastropub'] = df.cuisine_style.str.count('Gastropub')
df['bar'] = df['bar_s'] + df['pub'] + df['winebar'] + df['brew_pub'] + df['gastropub']
df = df.drop(['bar_s', 'pub', 'winebar', 'brew_pub', 'gastropub'], axis=1)

In [21]:
df['italian'] = df.cuisine_style.str.count('Italian')

In [22]:
df['pizza'] = df.cuisine_style.str.count('Pizza')

In [23]:
df['french'] = df.cuisine_style.str.count('French')

In [24]:
df['asian'] = df.cuisine_style.str.count('Asian')

In [25]:
df['spanish'] = df.cuisine_style.str.count('Spanish')

In [26]:
df['cafe'] = df.cuisine_style.str.count('Cafe')

In [27]:
df['american'] = df.cuisine_style.str.count('American')

In [28]:
df['british'] = df.cuisine_style.str.count('British')

In [29]:
df['fast_food'] = df.cuisine_style.str.count('Fast Food')
df['street_food'] = df.cuisine_style.str.count('Street Food')
df['delicatessen'] = df.cuisine_style.str.count('Delicatessen')
df['ff'] = df['fast_food'] + df['street_food'] + df['delicatessen']
df = df.drop(['fast_food', 'street_food', 'delicatessen'], axis=1)

In [30]:
df['international'] = df.cuisine_style.str.count('International')

In [31]:
df['seafood'] = df.cuisine_style.str.count('Seafood')

In [32]:
df['jap'] = df.cuisine_style.str.count('Japanese')
df['sushi'] = df.cuisine_style.str.count('Sushi')
df['japan'] = df['jap'] + df['sushi']
df = df.drop(['jap', 'sushi'], axis=1)

In [33]:
df['chinese'] = df.cuisine_style.str.count('Chinese')

In [34]:
df['portuguese'] = df.cuisine_style.str.count('Portuguese')

In [35]:
df['indian'] = df.cuisine_style.str.count('Indian')

In [36]:
df['other'] = 0
df.loc[(df['vegan'] == 0) & (df['european'] == 0) & 
    (df['glutenfree'] == 0) & (df['mediterranean'] == 0) &
    (df['bar'] == 0) & (df['italian'] == 0) &
    (df['pizza'] == 0) & (df['french'] == 0) &  
    (df['spanish'] == 0) & (df['asian'] == 0) &  
    (df['american'] == 0) & (df['cafe'] == 0) &  
    (df['british'] == 0) & (df['ff'] == 0) &  
    (df['international'] == 0) & (df['seafood'] == 0) &
    (df['japan'] == 0) & (df['chinese'] == 0) &  
    (df['portuguese'] == 0) & (df['indian'] == 0),
       'other'] = 1

In [37]:
df.fillna(0)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,...,american,british,ff,international,seafood,japan,chinese,portuguese,indian,other
0,id_5569,Paris,"European, French, International",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,id_1535,Stockholm,[a],1537.0,4.0,0,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,id_352,London,"Japanese, Sushi, Asian, Grill, Vegetarian Frie...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
3,id_3456,Berlin,[a],3458.0,5.0,0,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,id_615,Munich,"German, Central European, Vegetarian Friendly",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"Italian, Vegetarian Friendly, Vegan Options, G...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
39996,id_6340,Paris,"French, American, Bar, European, Vegetarian Fr...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
39997,id_1649,Stockholm,"Japanese, Sushi",1652.0,4.5,0,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
39998,id_640,Warsaw,"Polish, European, Eastern European, Central Eu...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


### 3. Ranking

Ординальный признак, отображающий место среди ресторанов своего города. Есть предположения, что это переработанный ключевой признак Rating, о чем свидетельствует довольно высокая корреляция. Для нормализации применил логарифмирование по основанию 2.

In [ ]:
def histogram(df):
    sns.set()
    f, axes = plt.subplots()
    sns.distplot(df)

In [ ]:
histogram(df.ranking)

In [ ]:
sns.jointplot(df.ranking, df.rating, data=df, kind = 'reg');

In [ ]:
correlation = df.corr()
sns.heatmap(correlation, cmap = 'coolwarm');

In [ ]:
correlation

In [38]:
df.ranking = np.log2(df.ranking)

### 4. Rating

Целевой параметр. Распределен нормально. Выбросов нет.

In [ ]:
histogram(df.rating)

### 5. Price Range

Ординальная переменная. Необходимо преобразовать данные. предположительно "$" - низкий уровень цен, "$$-$$$" - средний уровень цен, "$$$$" - высокий. Заменим на числа от 1 до 3, где 1 соответствует низкому уровню цен. Наибольшее число ресторанов относится к среднему ценовому диапазону (70%). Большое количество пропусков (35%). Данных для замены недостаточно, поэтому заменим пропуски средним значением по конкретному городу.

In [ ]:
df.price_range.value_counts()

In [39]:
df.price_range = df.price_range.replace('$', 1)
df.price_range = df.price_range.replace('$$ - $$$', 2)
df.price_range = df.price_range.replace('$$$$', 3)

In [40]:
city_price = {}
for city in cities:
    city_price[city] = df.price_range[df.city == city].mean()
for city in cities:
    mask = (df.city == city) & df.price_range.isna()
    df.loc[mask, 'price_range'] = city_price[city]

### 6. Number of Reviews

Числовой признак. Пустых значений нет. Среднее значение - 25. Большой разброс значений. Корреляция с ключевым параметром слабая. Выбросов достаточно много, чтобы их игнорировать. Выделю их в отдельную группу и введу категориальную переменную. Пропуски заменил на среднее значение.

In [41]:
mean_num_of_reviews = round(df.number_of_reviews\
                      .value_counts().mean())

In [42]:
df.number_of_reviews = df.number_of_reviews\
                          .fillna(mean_num_of_reviews)

In [43]:
histogram(df.number_of_reviews)

NameError: name 'histogram' is not defined

In [44]:
df.number_of_reviews.describe()

count    40000.000000
mean       118.542650
std        287.127603
min          2.000000
25%         10.000000
50%         28.000000
75%        105.000000
max       9660.000000
Name: number_of_reviews, dtype: float64

In [ ]:
df.number_of_reviews[df.number_of_reviews > 1000].hist()

In [ ]:
df.number_of_reviews[df.number_of_reviews > 3000].hist()

In [ ]:
sns.boxplot(df.number_of_reviews, color='yellow');

In [ ]:
def outliers_iqr(ys):
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]

In [ ]:
o = outliers_iqr(df.number_of_reviews)

In [ ]:
len(o)

In [45]:
median = df.number_of_reviews.median()
std = df.number_of_reviews.std()
perc25 = df.number_of_reviews.quantile(0.25)
perc75 = df.number_of_reviews.quantile(0.75)
lim_outliers = median + std*3
IQR = perc75 - perc25

In [46]:
def get_categ_variable(x):
    if x <= perc25:
        return 1
    if perc25 < x <= median:
        return 2
    if median < x <= perc75:
        return 3
    if perc75 < x <= lim_outliers:
        return 4
    if lim_outliers < x:
        return 5

In [47]:
df.number_of_reviews = df.number_of_reviews\
                           .apply(get_categ_variable)

In [48]:
df.number_of_reviews.value_counts()

1    10297
2     9961
3     9798
4     9123
5      821
Name: number_of_reviews, dtype: int64

In [49]:
df2 = pd.get_dummies(df.number_of_reviews)

In [50]:
df = df.merge(df2, left_index=True, right_index=True)

### 7. Reviews

In [52]:
df['date_of_review'] = df.reviews.str.findall('\d+/\d+/\d+')

In [53]:
df['date_1'] = df.date_of_review.map(lambda x: str(x)[2:12])
df['date_2'] = df.date_of_review.map(lambda x: str(x)[-12:-2])

In [54]:
df.date_1 = pd.to_datetime(df.date_1)
df.date_2 = pd.to_datetime(df.date_2)

In [55]:
df['range_date'] = abs(df.date_1 - df.date_2)

In [56]:
df['range_date'].fillna(pd.Timedelta('0 days'))

0         41 days
1        382 days
2          2 days
3          0 days
4        272 days
           ...   
39995     34 days
39996      9 days
39997   3127 days
39998     23 days
39999   1306 days
Name: range_date, Length: 40000, dtype: timedelta64[ns]

In [57]:
df['range_date'] = np.int64(df['range_date'])

In [ ]:
df.range_date.value_counts()

In [70]:
display(df)

,ranking,rating,price_range,number_of_reviews,vegan,european,glutenfree,mediterranean,bar,italian,...,japan,chinese,portuguese,indian,other,1,2,3,4,5
0,12.443462,3.5,2.000000,4,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,1,0
1,10.585901,4.0,1.943182,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
2,8.463524,4.5,3.000000,4,2.0,0.0,1.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0,0,0,0,1,0
3,11.755722,5.0,1.754799,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,1,0,0,0,0
4,9.278449,4.0,2.000000,3,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,8.965784,4.5,2.000000,3,2.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0
39996,12.630495,3.5,2.000000,4,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,0,1,0
39997,10.689998,4.5,1.943182,1,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.0,0,1,0,0,0,0
39998,9.324181,4.0,2.000000,3,1.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0,0,1,0,0


In [69]:
df = df.fillna(0)

In [58]:
df = df.drop(['restaurant_id', 'city', 'cuisine_style', 'reviews', 'url_ta', 'id_ta', 'date_of_review', 'date_1', 'date_2', 'cuisine_count', 'reviews_cuisine', 'range_date'], axis=1)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [71]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['rating'], axis = 1)
y = df['rating']

In [72]:
display(y)

0        3.5
1        4.0
2        4.5
3        5.0
4        4.0
        ... 
39995    4.5
39996    3.5
39997    4.5
39998    4.0
39999    3.0
Name: rating, Length: 40000, dtype: float64

In [73]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [74]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [75]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [76]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [77]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.31263982976190474
